In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import json
import sys
parent_dir = Path(sys.path[0])
data_dir = parent_dir.parent.parent / 'data'
if str(data_dir) not in sys.path:
    sys.path.append(str(data_dir))
from csd.csd import create_csd_dataset
from CMUdict.utils import CMUDict, VOCAB
# Inspired by https://www.kaggle.com/code/vitouphy/phoneme-recognition-with-wav2vec2

/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DATA_DIR already in sys.path ['/mnt/storage/projects/programming/song-transcription/code/transcription', '/home/victor/miniconda3/envs/song-transcription/lib/python311.zip', '/home/victor/miniconda3/envs/song-transcription/lib/python3.11', '/home/victor/miniconda3/envs/song-transcription/lib/python3.11/lib-dynload', '', '/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages', '/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages/huggingface_hub-0.19.4-py3.8.egg', '/tmp/tmpdt537dnt', '/mnt/storage/projects/programming/song-transcription/data']


In [2]:
csd_dataset = create_csd_dataset()
csd_dataset

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 456.45it/s]
Generating train split: 1728 examples [00:00, 155248.09 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'song_id', 'song_name', 'start', 'end', 'audio', 'phonemes', 'lyrics'],
        num_rows: 1382
    })
    test: Dataset({
        features: ['id', 'song_id', 'song_name', 'start', 'end', 'audio', 'phonemes', 'lyrics'],
        num_rows: 173
    })
    validation: Dataset({
        features: ['id', 'song_id', 'song_name', 'start', 'end', 'audio', 'phonemes', 'lyrics'],
        num_rows: 173
    })
})

In [3]:
csd_dataset['train'][0]

{'id': 710,
 'song_id': 'en022a',
 'song_name': 'Joy to the world',
 'start': 10.8662,
 'end': 15.205,
 'audio': {'path': '/mnt/storage/projects/programming/song-transcription/data/csd/segments/en022a_10.8662_15.205.wav',
  'array': array([-0.00221137, -0.0009494 ,  0.00193736, ...,  0.00469174,
          0.00313565,  0.        ]),
  'sampling_rate': 16000},
 'phonemes': 'L EH T EH EH V R IY HH AA AA R T P R IY P EH EH R HH IH M R UW UW M',
 'lyrics': 'Let every heart prepare him room'}

In [11]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained('working/', unk_token='[UNK]', pad_token='[PAD]', word_delimiter_token='|')
tokenizer.vocab_size

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


42

In [12]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16_000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [13]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='working/', vocab_size=42, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]', 'additional_special_tokens': [AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True)]}, clean_up_tokenization_spaces=True)

In [14]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    with processor.as_target_processor():
        batch["labels"] = processor(batch["phonemes"]).input_ids
    return batch
csd_dataset = csd_dataset.map(prepare_dataset)

Map: 100%|██████████| 173/173 [00:08<00:00, 19.52 examples/s]


In [15]:
import torch

from dataclasses import dataclass
from typing import Dict, List, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Copied from: https://www.kaggle.com/code/vitouphy/phoneme-recognition-with-wav2vec2s
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids)
    label_str = tokenizer.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {
        "wer": wer,
        "cer": cer
    }

/tmp/ipykernel_15214/2036523075.py:54: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [16]:
model = Wav2Vec2ForCTC.from_pretrained(
    "working/base2/final",
    attention_dropout=0.1,
    layerdrop=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.75,
    mask_time_length=10,
    mask_feature_prob=0.25,
    mask_feature_length=64,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True,
)
model.freeze_feature_extractor()
model

/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1925: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder)

In [17]:
training_args = TrainingArguments(
    output_dir='working/csd-model',
    group_by_length=True,
    per_device_train_batch_size=8,
    # gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    gradient_checkpointing=True,
    fp16=True,
    max_steps=10000,
    save_steps=500,  #100,
    eval_steps=500,
    logging_steps=100,
    learning_rate=3e-5,
    warmup_steps=2000,
    save_total_limit=3,
    load_best_model_at_end=True
)
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=csd_dataset['train'],
    eval_dataset=csd_dataset['validation'],
    tokenizer=processor.feature_extractor
)
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 369028 KiB | 369028 KiB | 369028 KiB |      0 B   |
|       from large pool | 368384 KiB | 368384 KiB | 368384 KiB |      0 B   |
|       from small pool |    644 KiB |    644 KiB |    644 KiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         | 369028 KiB | 369028 KiB | 369028 KiB |      0 B   |
|       from large pool | 368384 KiB | 368384 KiB | 368384 KiB |

In [18]:
trainer.train()

  0%|          | 0/10000 [00:00<?, ?it/s]/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
  1%|          | 100/10000 [02:13<3:59:31,  1.45s/it]

{'loss': 5.4498, 'learning_rate': 1.4250000000000001e-06, 'epoch': 0.58}


  2%|▏         | 200/10000 [04:15<2:39:05,  1.03it/s]

{'loss': 3.2715, 'learning_rate': 2.9250000000000004e-06, 'epoch': 1.16}


  3%|▎         | 300/10000 [06:16<1:45:16,  1.54it/s]

...

 25%|██▌       | 2500/10000 [54:38<1:40:36,  1.24it/s]

{'loss': 1.335, 'learning_rate': 2.81475e-05, 'epoch': 14.45}


                                                      
 25%|██▌       | 2500/10000 [54:55<1:40:36,  1.24it/s]

{'eval_loss': 0.9048542976379395, 'eval_wer': 0.6803827751196172, 'eval_cer': 0.2944602465122579, 'eval_runtime': 16.681, 'eval_samples_per_second': 10.371, 'eval_steps_per_second': 1.319, 'epoch': 14.45}


/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
 25%|██▌       | 2509/10000 [55:19<5:08:58,  2.47s/it] 